# Implementing ND Provers with ELPI

In this notebook we will explore how to implement natural deduction provers using ELPI.
For conciseness, we will only cover conjunction and implication.

**TODO: References and prerequisites**

In [1]:
elpi: logicsyntax

kind proposition type.
type and proposition -> proposition -> proposition.
type impl proposition -> proposition -> proposition.

% Some example propositions
type a proposition.
type b proposition.
type c proposition.

Successfully imported logicsyntax.elpi logicsyntax.elpi is the new default file for ELPI commands

## Naive attempt

In [2]:
elpi: naive

accumulate logicsyntax.

% we'll simply introduce a predicate indicating if a proposition is provable
type provable proposition -> prop.

% A∧B is provable if A is provable and B is provable (∧I)
provable (and A B) :- provable A, provable B.
% provable A :- provable (and A _B).   % ∧El
% provable B :- provable (and _A B).   % ∧Er

% A⇒B is provable if adding `provable A` to the context makes B provable (⇒I) 
provable (impl A B) :- provable A => provable B.
% provable B :- provable (impl A B), provable A.  % ⇒E

Successfully imported naive.elpi naive.elpi is the new default file for ELPI commands

In the naive implementation, the proof search diverges for the elimination rules (which is why they are commented out).
However, the prover can already find proofs that only require `∧I` and `⇒I`:

In [3]:
-- a ⇒ b ⇒ a∧b
query "provable (impl a (impl b (and a b)))"

Query succeeded

In [4]:
-- a ⇒ b ⇒ c
query "provable (impl a (impl b c))"

Query failed

In [5]:
-- a∧b⇒b∧a also fails because proving it requires the elimination rules that were commented out
query "provable (impl (and a b) (and b a))"

Query failed

## Limiting the Search Depth

To prevent the proof search from diverging, we can introduce another argument that limits the search depth.

In [6]:
elpi: depthlimited

accumulate logicsyntax.

type provable proposition -> int -> prop.

% ∧I
provable (and A B) Depth :- Depth > 0, RestDepth is Depth - 1,
    provable A RestDepth, provable B RestDepth.
% ∧El
provable A Depth :- Depth > 0, RestDepth is Depth - 1, provable (and A _B) RestDepth.
% ∧Er
provable B Depth :- Depth > 0, RestDepth is Depth - 1, provable (and _A B) RestDepth.

% ⇒I
provable (impl A B) Depth :- Depth > 0, RestDepth is Depth - 1,
    provable A _AnyDepth => provable B RestDepth.
% ⇒E
provable B Depth :- Depth > 0, RestDepth is Depth - 1,
    provable (impl A B) RestDepth, provable A RestDepth.

Successfully imported depthlimited.elpi depthlimited.elpi is the new default file for ELPI commands

In [7]:
-- a∧b⇒b∧a requires a depth of at least 3:
query "provable (impl (and a b) (and b a)) 1"
query "provable (impl (and a b) (and b a)) 2"
query "provable (impl (and a b) (and b a)) 3"
query "provable (impl (and a b) (and b a)) 4"

Query failed

Query failed

Query succeeded

Query succeeded

## Recording Proof Terms

So far, our `provable` predicate tells us if a proposition is a theorem or not, but it doesn't provide a proof.
To produce a natural deduction proof, we simply have to record the rule applications that led to a success.
The resulting proof terms are exactly the same (modulo syntax) as we used in MMT.

In [8]:
elpi: proofterms

% we have to introduce a type for proof terms
kind proofterm type.

% A proof of A and a proof of B give us a proof of A∧B (i.e. ⊢A ⟶ ⊢B ⟶ ⊢A∧B)
type andI proofterm -> proofterm -> proofterm.
type andEl proofterm -> proofterm.
type andEr proofterm -> proofterm.

type implI (proofterm -> proofterm) -> proofterm.    % (⊢A ⟶ ⊢B) ⟶ ⊢A⇒B
type implE proofterm -> proofterm -> proofterm.

Successfully imported proofterms.elpi proofterms.elpi is the new default file for ELPI commands

In [9]:
elpi: withproofterms

accumulate logicsyntax.
accumulate proofterms.

% We simply add one more argument to `provable` for recording the proof terms
type provable proposition -> int -> proofterm -> prop.

% ∧I
provable (and A B) Depth (andI ProofA ProofB):- Depth > 0, RestDepth is Depth - 1,
    provable A RestDepth ProofA, provable B RestDepth ProofB.
% ∧El
provable A Depth (andEl Proof) :- Depth > 0, RestDepth is Depth - 1, provable (and A _B) RestDepth Proof.
% ∧Er
provable B Depth (andEr Proof) :- Depth > 0, RestDepth is Depth - 1, provable (and _A B) RestDepth Proof.

% ⇒I
provable (impl A B) Depth (implI ProofFn):- Depth > 0, RestDepth is Depth - 1,
    pi proofA \ provable A _AnyDepth proofA => provable B RestDepth (ProofFn proofA).
% ⇒E
provable B Depth (implE ProofImpl ProofA) :- Depth > 0, RestDepth is Depth - 1,
    provable (impl A B) RestDepth ProofImpl, provable A RestDepth ProofA.

Successfully imported withproofterms.elpi withproofterms.elpi is the new default file for ELPI commands

###### Examples
Let's try to proof `a⇒a`. Recall that in MMT we would do this with `p : ⊢a⇒a ❘ = implI [proofOfA] proofOfA`.
In ELPI, the syntax for lambda functions is `x \ M` for `λx.M` (or `[x] M` in MMT).

In [10]:
query "provable (impl a a) 2 ProofTerm"

Query succeeded ProofTerm = implI c0 \ c0

In [11]:
-- a∧b⇒b∧a
query "provable (impl (and a b) (and b a)) 3 ProofTerm"

Query succeeded ProofTerm = implI c0 \ andI (andEr c0) (andEl c0)

In [12]:
-- setting a higher search depth, we usually get a longer proof term
query "provable (impl (and a b) (and b a)) 5 ProofTerm"

Query succeeded ProofTerm = andEl                (andI (implI c0 \ andI (andEr c0) (andEl c0))                  (andI (andI (implI c0 \ c0) (implI c0 \ c0))                    (andI (implI c0 \ c0) (implI c0 \ c0))))

## Adding Controller Code

Currently, using the prover is a bit unwieldy. It's particularly annoying that we have to manually pick the right search depth – if it is too small, no proof is found, and if it is too large, the search takes too long and the proof term will be unnecessarily complicated.
To remedy this, we can introduce some controller code that iteratively increments the search depth until a proof is found (iterative deepening).

In [13]:
elpi: controlledsearch

accumulate withproofterms.

type prove_helper proposition -> int -> int -> prop.
prove_helper Prop Depth MaxDepth :-
    Depth =< MaxDepth,
    print "Trying depth" Depth,
    provable Prop Depth ProofTerm,
    print "Found a proof:" ProofTerm.
prove_helper Prop Depth MaxDepth :-
    Depth < MaxDepth,
    NewDepth is Depth + 1,
    prove_helper Prop NewDepth MaxDepth.

type prove proposition -> prop.
prove Prop :- prove_helper Prop 0 5.

Successfully imported controlledsearch.elpi controlledsearch.elpi is the new default file for ELPI commands

In [14]:
query "prove (impl (and a b) (and b a))"

Trying depth 0 Trying depth 1 Trying depth 2 Trying depth 3 Found a proof: implI c0 \ andI (andEr c0) (andEl c0) Query succeeded